In [2]:
import mysql.connector
import pandas as pd
from datetime import date
import random as rnd
import datetime

In [5]:
con = mysql.connector.connect(host='127.0.0.1', database='library', port=3306, user='root', password='24682468Aa')

In [6]:
cur = con.cursor(dictionary=True)

# Проверить, есть ли у меня читательский билет

In [3]:
def check_id(phone):
    cur.execute("SELECT id_member FROM members\
                WHERE phone={}".format(phone))
    data=cur.fetchall()
    if data != []:
        print('Номер вашего читательского билета: {}'.format(data[0]['id_member']))
        return data[0]['id_member']
    else:
        print('Номер читательского билета для номера телефона {} не найден.\
        \nПопробуйте еще раз или получите читательский билет'.format(phone))

In [ ]:
name = str(input('Введите номер телефона: '))
member_id = check_id(name)

# Получить читательский билет

In [ ]:
def add_member(name, phone, email):
    reg_date = date.today().strftime("%Y-%m-%d")
    cur.execute("SELECT id_member, phone, email  FROM members \
                WHERE phone={} OR email LIKE '{}'".format(phone, email))
    data = cur.fetchall()
    
    if data == []:
        am = "INSERT INTO members (name, phone, email, reg_date) \
        VALUES ('{}', '{}', '{}', '{}')".format(name, phone, email, reg_date)
        cur.execute(am)
        con.commit()

        cur.execute("SELECT id_member FROM members ORDER BY id_member DESC LIMIT 1")
        data=cur.fetchall()
        print('\nВы успешно зарегестрированы!\
        \nНомер вашего читательского билета: {}'.format(data[0]['id_member']))
        return data[0]['id_member']
    else:
        
        print('\nПользователь с номером {} уже зарегестрирован.\
        \nНомер читательского билета: {}'.format(phone, data[0]['id_member']))
        return data[0]['id_member']

In [ ]:
name, phone, email  = input('Введите ФИО: '), str(input('Введите номер телефона: ')), input('Введите email: ')
member_id = add_member(name, phone, email)

# Поиск по авторам, жанру и названию

In [ ]:
def find_book(query):
    cur.execute("""SELECT * FROM (
                SELECT title, group_concat(DISTINCT name SEPARATOR ', ') as names, language, cat_name, pub_year, num_available, 
                books.id_book FROM authors, books, authors2books, category
                WHERE authors.id_author=authors2books.id_author 
                AND books.id_book=authors2books.id_book AND category.id_category=books.category
                GROUP BY id_book) as info
                WHERE info.title LIKE '%{}%'
                OR info.names LIKE '%{}%'
                OR info.cat_name LIKE '%{}%'
                """.format(query, query, query))
    data=cur.fetchall()
    if data==[]:
        print('По вашему запросу ничего не найдено')
    else:
        return pd.DataFrame.from_dict(data)

In [ ]:
query = str(input('Введите слов(а) для поиска: '))
df = find_book(query)
df

## Взять книгу

In [ ]:
def take_book(users_id):
    cur.execute("SELECT DISTINCT id_book, title FROM books LEFT JOIN loans \
                ON books.id_book=loans.book_id \
                WHERE id_book={} AND num_available > 0".format(df.iloc[users_id]['id_book']))
    avail=cur.fetchall()


    if avail == []: # check when book will be returned
        cur.execute("SELECT title, date_end FROM books LEFT JOIN loans \
                ON books.id_book=loans.book_id \
                WHERE id_book={} \
                ORDER BY date_end".format(df.iloc[users_id]['id_book']))
        data=cur.fetchall()
        print('К сожалению, сейчас нет доступных экземпляров книги "{}". \
        \nПредполагаемая дата возврата книги в библиотеку: {}'.format(data[0]['title'], data[0]['date_end']))

    else: # сделать запись в loan
        date_st = date.today()
        date_end = date_st + datetime.timedelta(days=30)

        date_end = date_end.strftime("%Y-%m-%d")
        date_st = date_st.strftime("%Y-%m-%d")
        id_book = str(avail[0]['id_book'])

        sss = "INSERT INTO loans (members_id, book_id, date_st, date_end) \
                VALUES ('{}')".format("', '".join([str(member_id), id_book, date_st, date_end])) 
        cur.execute(sss)
   
        # обновить avail в books
        sss = "UPDATE books SET num_available=num_available- 1\
                WHERE id_book={}".format(id_book, id_book) 
        cur.execute(sss)
        print('Вы успешно зарезервировали кингу "{}". \
        \nЕе необходимо вернуть до {}'.format(avail[0]['title'], date_end))

        con.commit()

In [ ]:
users_id = int(input('Введите номер нужной книги: '))
take_book(users_id)

# Проверить, какие книжки на руках и продлить их

In [ ]:
def check_my_books(member_id):
    cur.execute("SELECT title, date_end, id_book FROM books LEFT JOIN loans \
                ON books.id_book=loans.book_id \
                WHERE members_id={} \
                ORDER BY date_end".format(str(member_id)))
    data = cur.fetchall()
    if data != []:
        print("Ваши книги:")
        display(pd.DataFrame.from_dict(data))
    
        answer = input('Хотите продлить кингу? Да/Нет ')
        while answer.capitalize() == 'Да':
            book_id = int(input('Введите номер книги: '))
            update_date = """UPDATE loans SET date_end = DATE_ADD(date_end, INTERVAL 30 DAY)
                            WHERE book_id={} 
                            AND members_id={}""".format(str(data[book_id]['id_book']), member_id)
            cur.execute(update_date)
            
            cur.execute("""SELECT date_end FROM loans 
                            WHERE book_id={} 
                            AND members_id={}""".format(str(data[book_id]['id_book']), member_id))
            new_date = cur.fetchall()
            print("Дата возврата успешно изменена с {} на {}".format(data[book_id]['date_end'], new_date[0]['date_end']))
            answer = input('Хотите продлить кингу? Да/Нет ')
    else:
        print("Вы еще ничего не взяли из библиотеки")

In [ ]:
check_my_books(member_id)

# Проверить, есть ли неоплаченные штрафы

In [ ]:
def check_my_fees(member_id):
    cur.execute("SELECT DATEDIFF(CURDATE(), date_end) AS date_diff, \
                title, fee_amount FROM fees JOIN loans \
                ON fees.loan_id=loans.id_loan\
                JOIN books \
                ON books.id_book=loans.book_id\
                WHERE members_id={} AND\
                paid=0".format(str(member_id)))
    data = cur.fetchall()
    if data == []:
        print("У вас нет неоплаченных штрафов")
    else:
        for fee in data:
            print('Штраф за кннигу "{}" - {} р. Вы просрочили книгу на {} дня'.format(\
                                fee['title'], fee['fee_amount'], fee['date_diff']))

In [ ]:
check_my_fees(member_id)

Удаление

In [ ]:
cur.execute("SELECT id_loan FROM loans ORDER BY id_loan DESC LIMIT 5")
cur.fetchall()

In [ ]:
sss = "DELETE FROM loans ORDER BY id_loan DESC LIMIT 1" 
cur.execute(sss)
# con.commit()

# Запросы

1. топ-5 авторов по полулярности у читателей

In [7]:
cur.execute("""WITH full_authors AS (SELECT title, 
            GROUP_CONCAT(name SEPARATOR ', ') as cn_names, books.id_book
            FROM books, authors2books, authors
            WHERE 
            authors2books.id_author=authors.id_author
            AND authors2books.id_book=books.id_book
            GROUP BY books.id_book) 
            
            SELECT  cn_names, COUNT(DISTINCT(id_loan)) as cnt
            FROM loans, full_authors
            WHERE loans.book_id=full_authors.id_book
            GROUP BY cn_names
            ORDER BY cnt DESC
            LIMIT 5""")
data=cur.fetchall()
pd.DataFrame.from_dict(data)

,cn_names,cnt
0,"Ганс Андерсен, Эрнст Гофман, Гримм Якоб и Виль...",5
1,Себастьян Перез,4
2,Сергей Тармашев,4
3,Виктория Кирдий,3
4,Леонид Парфенов,3


2. сумма неоплаченных штрафов по пользователю

In [8]:
cur.execute("""SELECT name, SUM(fee_amount) as total_fee
                FROM loans JOIN fees ON loans.id_loan=fees.loan_id
                JOIN members ON members.id_member=loans.members_id
                WHERE paid=0
                GROUP BY members_id""")
data=cur.fetchall()
pd.DataFrame.from_dict(data)

,name,total_fee
0,Воробьева Мадина Денисовна,53700
1,Королев Павел Андреевич,35450
2,Смирнова Кристина Александровна,39700
3,Яковлев Арсений Константинович,54600
4,Карпов Демид Глебович,3150
5,Кузина Таисия Артёмовна,22750
6,Черная Ника Романовна,5750
7,Кузнецова Вероника Викторовна,39800


3. люди, у которых неоплаченный штраф выше среднего по неоплаченным

In [9]:
cur.execute("""SELECT name, fee_amount FROM fees 
                JOIN loans ON fees.loan_id=loans.id_loan 
                RIGHT JOIN members ON members.id_member=loans.members_id
                WHERE paid=0 
                AND fee_amount > (
                SELECT ROUND(AVG(fee_amount)) FROM fees WHERE paid=0)
                ORDER BY fee_amount DESC""")
data=cur.fetchall()
pd.DataFrame.from_dict(data)

,name,fee_amount
0,Воробьева Мадина Денисовна,53700
1,Кузнецова Вероника Викторовна,39800
2,Королев Павел Андреевич,35450
3,Яковлев Арсений Константинович,33850
4,Смирнова Кристина Александровна,28250


In [10]:
cur.close()
con.close()